In [1]:
# prompt: a machine learningmodel read a file from csv

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
filename = '/content/GOOGL (1).csv'

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(filename)

# Train a machine learning model using the data


# Make predictions using the trained model
# ...


In [3]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096
1,2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343
2,2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126
3,2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337
4,2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602


In [2]:
training_set=data.iloc[:,1:2].values
print(training_set)
print(training_set.shape)

[[  50.050049]
 [  50.555557]
 [  55.430431]
 ...
 [2722.030029]
 [2774.050049]
 [2784.      ]]
(4431, 1)


In [18]:
# prompt: import sklearn minmaxscaler with featurerange 0,1 and transform the training set to this minmax accordingly

from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

training_set_scaled

array([[1.36253641e-04],
       [3.06152003e-04],
       [1.94456941e-03],
       ...,
       [8.98173517e-01],
       [9.15657150e-01],
       [9.19001272e-01]])

In [4]:

X_train = []
y_train = []
for i in range(60,4431):
  X_train.append(training_set_scaled[i-60:i,0])
  y_train.append(training_set_scaled[i,0])
X_train=np.array(X_train)
y_train=np.array(y_train)

print(X_train.shape)
print(y_train.shape)


(4371, 60)
(4371,)


In [5]:
# prompt: reshape the x train and print the x train shape

X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)


(4371, 60, 1)


In [6]:
# prompt: import keras.models required for machine learning prediction sqntl,lstm,drop,dense

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM


In [7]:
regressor=Sequential()

regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

In [30]:
# prompt: compile the regressor with adam and epoch of 100 with batch size 32 and loss mse use fit

regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(X_train, y_train, batch_size=32, epochs=100)


Epoch 1/100
137/137 [==============================] - 29s 141ms/step - loss: 5.7341e-04
Epoch 2/100
137/137 [==============================] - 18s 130ms/step - loss: 5.2583e-04
Epoch 3/100
137/137 [==============================] - 17s 124ms/step - loss: 6.1153e-04
Epoch 4/100
137/137 [==============================] - 18s 131ms/step - loss: 5.4732e-04
Epoch 5/100
137/137 [==============================] - 17s 121ms/step - loss: 6.2107e-04
Epoch 6/100
137/137 [==============================] - 17s 124ms/step - loss: 4.4928e-04
Epoch 7/100
137/137 [==============================] - 18s 127ms/step - loss: 5.1014e-04
Epoch 8/100
137/137 [==============================] - 17s 123ms/step - loss: 5.0136e-04
Epoch 9/100
137/137 [==============================] - 17s 127ms/step - loss: 5.0239e-04
Epoch 10/100
137/137 [==============================] - 17s 127ms/step - loss: 5.0264e-04
Epoch 11/100
137/137 [==============================] - 18s 129ms/step - loss: 5.6846e-04
Epoch 12/100
137/13

In [31]:
dataset_test=pd.read_csv("/content/GOOGL (1).csv")
actual_price=dataset_test.iloc[:,1:2].values

In [32]:
dataset_total=pd.concat((data['Open'],dataset_test['Open']),axis=0)
inputs=dataset_total[len(dataset_total)-len(dataset_test)-60:].values

inputs=inputs.reshape(-1,1)
inputs=sc.transform(inputs)

X_test = []
for i in range(60,80):
  X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [33]:
predicted_stock_price=regressor.predict(X_test)
predicted_stock_price= sc.inverse_transform(predicted_stock_price)

1/1 [==============================] - 2s 2s/step


In [ ]:
# Assuming 'predicted_stock_price' contains your predicted values
plt.plot(actual_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
